<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Funk SVD RMSE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, funksvd
from lenskit.algorithms.funksvd import FunkSVD
from lenskit.metrics.predict import rmse
import pandas as pd
import numpy as np
import ipympl

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # train the algorithm with the training dataset
    # Now we run the recommender:
    preds = batch.predict(fittable, test)
    # add the algorithm name for analyzability
    return preds

In [5]:
def model_trainer(features,lrate,reg):
    all_recs = []
    test_data = []
    features=int(features)
    funksvd = FunkSVD(features=features, iterations=100, lrate=lrate, reg=reg, damping=5, range=None, bias=True) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('FunkSVD', funksvd, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)

    metric = rmse(all_recs['prediction'], all_recs['rating'])
    return metric

In [6]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    lrate = params['lrate']
    reg = params['reg']
    metric = model_trainer(features,lrate,reg)
    print([features,lrate,reg,metric],',')
    return {'loss': metric, 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100),
       'lrate': hp.uniform('lrate', 0.0005, 0.005),
       'reg': hp.uniform('reg',0.005,0.05)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[36.324235491044504, 0.001997981153218362, 0.049453639967432905, 0.91406382414893]                                     
,                                                                                                                      
[99.84580099901486, 0.003794247454879051, 0.04692541302341471, 0.915337540270567]                                      
,                                                                                                                      
[47.81754861150867, 0.0020373992313963618, 0.03750611097510557, 0.9183842708517814]                                    
,                                                                                                                      
[90.50619848446688, 0.0025170301810330687, 0.02199381871802702, 0.909251806231808]                                     
,                                                                                                                      
[19.904079320993034, 0.00320965988053935

[90.65567376904087, 0.0024121062630496977, 0.03647492345107432, 0.9049168891771912]                                    
,                                                                                                                      
[68.61471157826666, 0.0020256547741441563, 0.02309169374702803, 0.905453103144743]                                     
,                                                                                                                      
[95.41680994530437, 0.0015154413343596998, 0.01821549342493515, 0.9223335297098966]                                    
,                                                                                                                      
[76.67481518251564, 0.002667478633944268, 0.039686761451970254, 0.8946444887971265]                                    
,                                                                                                                      
[61.282886379273776, 0.00370751067750865

In [9]:
metric = pd.DataFrame(array)
metric.columns = ['features','learning rate','reg','RMSE']
metric.head(5)

,features,learning rate,reg,RMSE
0,36.324235,0.001998,0.049454,0.914064
1,99.845801,0.003794,0.046925,0.915338
2,47.817549,0.002037,0.037506,0.918384
3,90.506198,0.002517,0.021994,0.909252
4,19.904079,0.003210,0.025331,0.933796


In [10]:
metric.sort_values(by=['RMSE'],ascending=True).head(10)

,features,learning rate,reg,RMSE
71,76.674815,0.002667,0.039687,0.894644
77,71.646864,0.003174,0.044741,0.898392
75,71.951324,0.002562,0.035766,0.900001
85,50.466828,0.003279,0.049921,0.901277
57,70.557025,0.002312,0.024626,0.901807
39,80.826402,0.002132,0.033739,0.902451
49,94.790331,0.001895,0.029737,0.903135
34,86.959871,0.002658,0.038023,0.903166
54,95.507469,0.002057,0.026776,0.903556
65,78.814275,0.002238,0.031052,0.904017


In [2]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Algoritmos\\FunkSVD\\results\\metric_RMSE.csv')
metric

,features,learning rate,reg,RMSE
0,36.324235,0.001998,0.049454,0.914064
1,99.845801,0.003794,0.046925,0.915338
2,47.817549,0.002037,0.037506,0.918384
3,90.506198,0.002517,0.021994,0.909252
4,19.904079,0.003210,0.025331,0.933796
...,...,...,...,...
95,97.390066,0.002900,0.048122,0.908302
96,32.988017,0.003810,0.037100,0.927885
97,89.876777,0.004624,0.040086,0.941086
98,48.131510,0.003352,0.041714,0.913568


In [4]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['learning rate']
z = metric['features'] 
c = metric['RMSE']


fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.hsv())
ax.set_xlabel('reg')
ax.set_ylabel('learning rate')
ax.set_zlabel('features')
fig.colorbar(img)
plt.show()

In [30]:
import ipyvolume as ipv
from matplotlib import cm
colormap = cm.hot


x = metric['reg']
y = metric['learning rate']
z = metric['features'] 
c = metric['RMSE']
color = colormap(c)
fig = ipv.figure()
ipv.style.use(['seaborn-darkgrid'])
ipv.pylab.xlim(0, 0.05)
ipv.pylab.ylim(0, 0.005)
ipv.pylab.zlim(0, 100)
ipv.scatter(x,y,z,color = color, marker='sphere')

ipv.pylab.xlabel('Reg')
ipv.pylab.ylabel('Learning rate')
ipv.pylab.zlabel('Number of features')
ipv.show()